## 교차 검증과 그리드 서치
- 테스트 세트를 사용해 자꾸 성능을 확인하게 되면 결국 모델은 테스트 세트에 맞추게 되는 셈이다.
- 테스트로 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하면 안된다.(모델을 만들고 마지막에 한번 사용하는게 좋음)


## 검증 세트
- 테스트 세트를 사용하지 않고 이를 측정하는 방법은 훈련 세트를 또 나누는것이다.
- 80%의 훈련세트를 60%로, 나머지 20%를 검증 세트로 분류한다.

In [103]:
import pandas as pd
wine = pd.read_csv("https://bit.ly/wine_csv_data")
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [104]:
# 특성과 결과값 분할
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()

In [105]:
# 훈련과 테스트 세트 분할
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, random_state=42)

In [106]:
# train_input과 train_target을 다시 train_test_split() 함수에 넣어 훈련세트와 검증세트로 나눈다.
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [107]:
# 구조 확인
print(sub_input.shape, val_input.shape)

(3897, 3) (975, 3)


In [109]:
# 모델 학습
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

# 확실히 과대적합되어있다.

0.997947138824737
0.8420512820512821


## 교차 검증
- 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어진다.
- 그렇다고 검증 세트를 너무 조금 떼어 놓으면 점수가 들쭉날쭉하고 불안정할것이다.
- 이럴 때 교차검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

- 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다.
- 그 다음에 이 점수를 평균하여 최종 검증 점수를 얻는다.
  - 예) n(3, 5, 10 ...)-폴드 교차 검증

- 사이킷런에는 cross_validate()라는 교차 검증 함수를 사용한다.
  - 평가할 모델 객체를 첫 번째 매개변수로 전달, 그 다음 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 함수에 전달한다.

In [116]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02053761, 0.01160741, 0.01466846, 0.01109362, 0.00704885]), 'score_time': array([0.        , 0.        , 0.00051498, 0.00202203, 0.        ]), 'test_score': array([0.85128205, 0.84820513, 0.8788501 , 0.85112936, 0.84394251])}


- fit_time: 모델 훈련 시간
- score_time: 검증하는 시간
- test_score: 교차 검증의 최종 점수

In [119]:
import numpy as np
print(np.mean(scores['test_score']))

0.8546818301479492


- cross_validate() 함수는 훈련 세트를 섞어 폴드를 나누지 않는다.
- 우리는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비해서 상관은 없다.
- 그렇지만 만약 훈련 세트를 섞으려면 분할기를 지정해야한다.
  - 보통 회귀 문제일땐 KFold 분할기, 분류 문제일땐 StratifiedKFold를 사용한다.

In [122]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8546818301479492


In [124]:
# 10 폴드 교차 검증은? (기본이 5)
from sklearn.model_selection import StratifiedKFold

splittr = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8546818301479492


## 하이퍼파라미터 튜닝
- 모델 파라미터: 머신러닝 모델이 학습하는 파라미터
- 하이퍼파라미터: 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터
  - max_depth, min_samples_split과 같은 매개변수 존재
  - 두 값을 같이 조율해 최적화된 값을 찾는다

### 그리드서치
- 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다.
  - 별도로 cross_validate()를 호출할 필요 없음

In [127]:
# min_impurity_decrease 매개변수의 최적값을 찾아보자
from sklearn.model_selection import GridSearchCV

# 5개의 값 미리 지정
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# 결정 트리 객체 생성시 파라미터 바로 전달
# n_jobs는 cpu의 실행 코어 수 지정, -1은 모든 코어이다
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)

In [129]:
# 그리드 서치 수행
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [130]:
# GridSearchCV는 최적의 매개변수가 적용된 dt 모델을 자동으로 재학습시켜준다.
# 그 dt 객체는 gs의 best_estimator_에 저장되어있다.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9137931034482759


In [131]:
# 서치로 찾은 최적의 매개변수
print(gs.best_params_)

{'min_impurity_decrease': 0.0003}


In [132]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'에 들어있다.
print(gs.cv_results_['mean_test_score'])

[0.86843111 0.86925267 0.87315179 0.87212531 0.87130627]


- 결정 트리에서 mean_impurity_decrease는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다.
- 여기에 max_depth로 트리 깊이를 제한하고, min_samples_split으로 노드를 나누기 위한 최소 샘플 수도 골라본다.
- dt의 하이퍼파라미터 목록: https://chatgpt.com/c/67db8929-9d28-8011-8cd3-df6bd6e97b62

In [134]:
# numpy의 arange함수와 파이썬의 range 함수 사용
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 200, 10)
         }

In [ ]:
# GridSearchCV 모델 생성 및 학습
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

In [ ]:
# 최적의 파라미터 확인
print(gs.best_params_)

In [ ]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

## 랜덤 서치
- 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 너무 많은 매개 변수 조건으로 인해 어려울 수 있다.
- 랜덤 서치에는 매개변수 값의 목록을 전달하는것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.
- 싸이파이를 이용한다.
  - 싸이파이: 파이썬의 핵심 과학 라이브러리 중 하나. 적분, 보간, 선형대수 등 지원
- stats 서브 패키지에 있는 uniform(실숫값)과 randint(정숫값) 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다.
  - 이를 "균등 분포에서 샘플링한다"라고 말한다.

In [ ]:
from scipy.stats import uniform, randint

rgen = randint(0, 10)
rgen.rvs(10)

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

In [ ]:
ugen = uniform(0, 1)
ugen.rvs(10)

### 난수 발생기랑 유사하게 생각하면 된다.
- randint와 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 찾으라고 명령할 수 있다.

In [ ]:
# 탐색할 매개변수 지정
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25)
         }

In [ ]:
# 학습
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

In [ ]:
# 최적의 매개변수 조합 출력
print(gs.best_params_)

In [ ]:
# 최고의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
# 최적의 모델은 best_estimator에 저장되있다. 꺼내와서 score를 확인해보자
dt = gs.best_estimator_
print(dt.score(test_input, test_target))